# SOBEL EDGE DETECTION

In [1]:
import numpy as np
import cv2

def sobel_edge_detection(image):
    # Converted colored image to gray scale based on the luminosity values of the human eyes.
    
    r, g, b = image[:,:,0], image[:,:,1], image[:,:,2]
    gray_image= 0.21 * r + 0.72 * g + 0.07 * b
    
    # Applied Gaussian blur to the grayscale image for reducing the noises.
    blurred_image = cv2.GaussianBlur(gray_image, (5,5), 0)
    
    # Defined Sobel kernels for calculating the horizontal and vertical gradients.
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])

    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    
    # Convolved the blurred image with the Sobel kernels:
    gradient_x = cv2.filter2D(gray_image, -1, sobel_x)
    gradient_y = cv2.filter2D(gray_image, -1, sobel_y)
    
    # Calculated the magnitude of gradients.
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    
    
    # Normalized the gradient magnitude to the range [0, 255].
    gradient_magnitude = np.uint8(gradient_magnitude / np.max(gradient_magnitude) * 255)
    
    enhanced_gradient = cv2.convertScaleAbs(gradient_magnitude, alpha=2.2, beta=1) #Enhanced edges for better visualization.

    # Apply a threshold to further enhance edges
    _, thresholded_gradient = cv2.threshold(enhanced_gradient, 21, 255, cv2.THRESH_BINARY) # Applied thresholding.

    return thresholded_gradient

# Resized the image for faster computation.
image = cv2.imread('table.png')
width = 400
height = 300

resized_image = cv2.resize(image, (width,height))
    
# Performed Sobel edge detection
edge_image = sobel_edge_detection(resized_image)

# Saved the result
cv2.imwrite('edge.png', edge_image)

True

# HOUGH TRANSFORMATION

In [2]:
# Firstly Done Hough transform from python's inbuilt function.

Hough_lines = cv2.HoughLines(edge_image, rho=1, theta=np.pi/180, threshold=292)

# Drawn detected lines on the original image.

output_image = np.copy(resized_image)

if Hough_lines is not None:
    for line in Hough_lines:
        rho, theta = line[0]
        A = np.cos(theta)
        B = np.sin(theta)
        x0 = A * rho
        y0 = B * rho
        x1 = int(x0 + 500 * (-B))
        y1 = int(y0 + 500 * (A))
        x2 = int(x0 - 500 * (-B))
        y2 = int(y0 - 500 * (A))
        cv2.line(output_image, (x1, y1), (x2, y2), (0, 0, 255), 2) # Drwan output lines with red color and thickness=2.

# Saved the output image.
cv2.imwrite('Hough_lines.png', output_image)

True

# HOUGH TRANSFORMATION FROM SCRATCH

In [3]:
import cv2
import numpy as np

def hough_transform(image, threshold):
    # Defined the Hough space in (rho, theta) for getting various rho and theta for a point in the image space.
    rows, cols = image.shape
    diag_length = int(np.ceil(np.sqrt(rows ** 2 + cols ** 2))) # Calculated diagonal length.
    rhos = np.linspace(-diag_length, diag_length, diag_length * 2) # Got 2*D rhos from -D to D.
    thetas = np.deg2rad(np.arange(-90, 90, 1)) # Getting vaious thetas from -90 to 90 and converted them to radians.

    # Initializing the accumulator array for setting the threshold to the values of Rho and theta.
    accumulator = np.zeros((2 * diag_length, len(thetas)), dtype=np.uint64)
    # In the accumulator array I represented rho by rows and thetas by columns. 
    
    # Storing the edge points indices in the variable edge points.
    edge_points = np.argwhere(image > 0)

    # Looping through edge points and calculating the values in accumulator array for each edge point.
    for y, x in edge_points:
        for idx, theta in enumerate(thetas):
            rho = int(x * np.cos(theta) + y * np.sin(theta))
            accumulator[rho + diag_length, idx] += 1  # Used rho+D to avoid negative values of rho. 

    # Find lines with votes above threshold
    lines = np.argwhere(accumulator >= threshold)    # Storing indices where the value in the accumulator array is greater than the threshold.

    return lines

# Read the edge image
edge_image = cv2.imread('edge.png', cv2.IMREAD_GRAYSCALE)

# Applying the hough transform
lines = hough_transform(edge_image, threshold=295)



image = cv2.imread('table.png')

# Visualizing the detected lines
for rho, theta in lines:
    # Transform the polar coordinates to Cartesian coordinates
    
    A = np.cos(theta)
    B = np.sin(theta)
    x0 = A * rho
    y0 = B * rho
    
    # Extend the lines to the edges of the image
    x1 = int(x0 + 1000 * (-B))
    y1 = int(y0 + 1000 * (A))
    x2 = int(x0 - 1000 * (-B))
    y2 = int(y0 - 1000 * (A))
    
    # Draw the lines on the original image
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 1) # Drawn lines with red color with thickness 1.

# Save the image with detected lines
cv2.imwrite('Hough_lines_from_scratch.png', image)


True